In [1]:
import requests
from selenium import webdriver
from pyquery import PyQuery as pq
from selenium.webdriver.common.keys import Keys
from pyquery import PyQuery as pq

import time
from time import sleep

In [2]:
driver = webdriver.PhantomJS(executable_path=r"D:\PyProjects\projcrawler\phantomjs-2.1.1-windows\bin\phantomjs.exe")

C:\Users\User\anaconda3\envs\py38\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [3]:
bidSystemUrl = "https://web.pcc.gov.tw/tps/pss/tender.do?method=goSearch&searchMode=common&searchType=basic"
browse = driver.get(bidSystemUrl)
homepage = driver.page_source

In [11]:
query = driver.find_element_by_name("tenderName")
query.clear()
query.send_keys("邊坡")
queryCate = driver.find_element_by_css_selector("#radProctrgCate1")
queryCate.click()
query.send_keys(Keys.RETURN)

In [12]:
sleep(1) #等頁面回傳
bidpage = driver.page_source

In [21]:
soup = BeautifulSoup(bidpage, 'html.parser')

In [30]:
soup.find("td", bgcolor="#98BADC").find_all("table")[1]

<table border="0" cellpadding="0" cellspacing="0" style="word-break:break-all" width="100%">
<tbody><tr>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="5%">項次 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="15%">機關 <br/>名稱 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="26%">標案案號 <br/>標案名稱 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="5%">傳輸 <br/>次數 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="10%">招標 <br/>方式 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="8%">採購 <br/>性質 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="9%">公告 <br/>日期 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="9%">截止 <br/>投標 </th>
<th align="center" bgcolor="#E0EDF1" class="T12b" nowrap="" width="13%">預算 <br/>金額 </th>
</tr>
<tr bgcolor="#FFFFFF" onmouseout="outcss(this,'#FFFFFF');" onmouseover="overcss(this);">
<!-- 項次 

In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

import json

from datetime import datetime, timedelta
import os

In [2]:
def get_proj_list_by_date(date):
    unit_keywords = ['宜蘭', '羅東', '第四區養護工程處']
    date_url = f"https://pcc.g0v.ronny.tw/api/listbydate?date={date}"
    
    r = requests.get(date_url)
    d = json.loads(r.text)
    p = [i for i in d['records'] if (i['brief']['type'] == '公開招標公告') and ('工程類' in i['brief']['category'])]
    
    projs = []
    for k in unit_keywords:
        for i in p:
            try:
                if k in i['unit_name']:
                    projs.append(i)
            except:
                continue
    return projs

def get_proj_info(unit_id, job_number):
    proj_url = f"https://pcc.g0v.ronny.tw/api/tender?unit_id={unit_id}&job_number={job_number}"
    r = requests.get(proj_url)
    d = json.loads(r.text)
    return {
        "unit_name": d['records'][0]['unit_name'], 
        "proj_name": d['records'][0]['brief']['title'], 
        "proj_type": d['records'][0]['brief']['type'], 
        "proj_category": d['records'][0]['brief']['category'], 
        "proj_budget": d['records'][0]['detail']['採購資料:預算金額'], 
        "proj_loc": d['records'][0]['detail']['其他:履約地點'], 
        "proj_pub_time":d['records'][0]['detail']['招標資料:公告日'], 
        "proj_end_time":d['records'][0]['detail']['領投開標:截止投標'], 
        "proj_open_time":d['records'][0]['detail']['領投開標:開標時間'], 
        "url": d['records'][0]['detail']['url']
           }

In [3]:
# Monday is 0, Sunday is 6
weekday = datetime.today().weekday()
day_list = [i.strftime('%Y%m%d') for i in pd.date_range(datetime.today()-timedelta(days=7+weekday), datetime.today()-timedelta(days=2+weekday))][:-1]
day_list

['20210726', '20210727', '20210728', '20210729', '20210730']

In [4]:
proj_df = pd.DataFrame()
for date in day_list:
    projs = get_proj_list_by_date(date)
    for proj in projs:
        temp_df = pd.DataFrame()
        proj_info = get_proj_info(proj['unit_id'], proj['job_number'])
        for k, v in proj_info.items():
            temp_df[k] = [v]
        proj_df = pd.concat([proj_df, temp_df], ignore_index=True)
proj_df.drop_duplicates(keep='last', ignore_index=True, inplace=True)
proj_df.rename(columns={"unit_name":"機關名稱", "proj_name":"標案名稱", "proj_type":"招標方式", "proj_category":"標的分類", 
                        "proj_budget":"預算金額", "proj_loc":"履約地點", "proj_pub_time":"公告日", "proj_end_time":"截止投標", 
                        "proj_open_time":"開標時間"}, inplace=True)

In [5]:
saved_name = os.path.join("c:", "Users", "User", "Desktop", "標案資料", f"{datetime.today().strftime('%Y%m%d')}.xlsx")
proj_df.to_excel(saved_name, index=None)